### ===Task===

Your work: Let's modify the above scratch code:
- Notice that if <code>err</code> = 0, then $\alpha$ will be undefined, thus attempt to fix this by adding some very small value to the lower term
- Notice that sklearn version of AdaBoost has a parameter <code>learning_rate</code>.  This is in fact the $\frac{1}{2}$ in front of the $\alpha$ calculation.  Attempt to change this $\frac{1}{2}$ into a parameter called <code>eta</code>, and try different values of it and see whether accuracy is improved.  Note that sklearn default this value to 1.
- Observe that we are actually using sklearn DecisionTreeClassifier.  If we take a look at it closely, it is actually using weighted gini index, instead of weighted errors that we learn above.   Attempt to write your own class of <code>class Stump</code> that actually uses weighted errors, instead of weighted gini index
- Put everything into a class

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [2]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=500, random_state=1)
y = np.where(y==0,-1,1)  #change our y to be -1 if it is 0, otherwise 1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [3]:
np.set_printoptions(suppress=True)

In [13]:
class DecisionStump():
    def __init__(self):
        # Determines whether threshold should be evaluated as < or >
        self.polarity = 1
        self.feature_index = None
        self.threshold = None
        # Voting power of the stump
        self.alpha = None

In [14]:
class Stump():
    def __init__(self, num_stumps=5, eta=0.5):
        self.num_stumps = num_stumps
        self.eta = eta
        self.models = [DecisionStump() for _ in range(self.num_stumps)]
        
    def fit(self, X, y): #<----X_train, y_train
        m,n = X.shape
        
        #initially, we set our weight to 1/m
        W = np.full(m, 1/m)
        
        #Loop over all models we created
        for clf in self.models:    
    
          #highest possible value so the first is always less than this
            min_err = np.inf
            
            #Loop for all features and all thresholds in X and find its error             
            for feature in range(n):
                #sort the values so we can divide by 2 to get midpoinst
                feature_vals = np.sort(X[:, feature])
                
                #get all the thresholds
                threshold_list = (feature_vals[:-1] + feature_vals[1:])/2
                
                
                for threshold in threshold_list:
                    #Try both polarity becuase we set all to 1 at first
                    #1 or -1 could on either side of the threshold
                    for polarity in [1, -1]:
                        yhat = np.ones(len(y)) #set all to 1
                        
                        #polarity=1 rule 
                        #separate the two classes at given threshold
                        yhat[polarity * X[:, feature] < polarity * threshold] = -1  
                        err = W[(yhat != y)].sum()
                                        
                        #to save the best stump
                        if err < min_err:
                            clf.polarity = polarity
                            clf.threshold = threshold
                            clf.feature_index = feature
                            min_err = err
        
            #only the best stump(threshold) for the feature will be saved in the Decision Stump class
            #we calculate its alpha, and reweight samples
            
            lap = 1e-5 #to prevent division by zero
            term2 = np.log ((1 - min_err) / (min_err + lap))
            clf.alpha = self.eta * term2  
            
            W = W * np.exp(-clf.alpha * y * yhat)/sum (W)
        
    def predict(self, X):
        m, n = X.shape
        yhat = np.zeros(m)
        #for clf in self.clf_list:
        for clf in self.models:
            pred = np.ones(m) #set all to 1
            pred[clf.polarity * X[:, clf.feature_index] < clf.polarity * clf.threshold] = -1  #polarity=1 rule
            yhat = yhat + clf.alpha * pred

        return np.sign(yhat)

In [15]:
model = Stump(num_stumps=10, eta = 0.6)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.72      0.99      0.83        79
           1       0.98      0.56      0.71        71

    accuracy                           0.79       150
   macro avg       0.85      0.78      0.77       150
weighted avg       0.84      0.79      0.78       150

